In [1]:
import numpy as np
import cv2
import time
import datetime
import os
import shutil
import dill
import pandas as pd
import warnings
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from keras.utils import Sequence, to_categorical
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.models import Sequential, load_model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.neighbors.kde import KernelDensity
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

Using TensorFlow backend.


In [2]:
class VideoStream:
    
    def __init__(self, filepath):
        self.cap = cv2.VideoCapture(filepath)
        if not self.cap.isOpened():
            raise Exception('Video stream doesn\'t open!')
    
    def __enter__(self):
        return self.cap
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cap.release()

In [3]:
class ShortVideoWarning(UserWarning):
    
    def __init__(self, message):
        super().__init__()
        self.message = message
        
    def __str__(self):
        return self.message

In [180]:
def read_ids(filename):
    ids = []
    classes = []
    with open(filename) as f:
        next(f)
        for line in f:
            line = line.strip()
            id_, class_ = line.split(',')
            ids.append(id_)
            classes.append(class_)
    return ids, classes

In [181]:
train_ids, train_classes = read_ids('../IDs/train.csv')
valid_ids, valid_classes = read_ids('../IDs/valid.csv')
test_ids, test_classes = read_ids('../IDs/test.csv')

In [237]:
class ImageSequenceGenerator(Sequence):
    
    def __init__(
        self,
        videos,
        ids,
        classes,
        class_names,
        target_size=(224, 224),
        fps=8,
        sequence_time=3,
        shift_time=1,
        batch_size=8,
        shuffle=True,
        seed=None,
        fit_eval=True
    ):
        self.videos = videos
        self.ids = ids
        self.classes = classes
        self.class_names = class_names
        self.target_size = target_size
        self.fps = fps
        self.sequence_time = sequence_time
        self.shift_time = shift_time
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.fit_eval = fit_eval
        
        self.timesteps = self.fps * self.sequence_time
        self.shift_frames = self.fps * self.shift_time
        
        self.skip_id = []
        self.start_positions = []
        self.indexes = None
        
        self.__check_videos_total_time()
        self.__make_start_positions()
        self.on_epoch_end()

    def __check_videos_total_time(self):
        all_ids = os.listdir(self.videos)
        self.skip_id = []
        for id_ in all_ids:
            video_path = os.path.join(self.videos, id_)
            with VideoStream(video_path) as cap:
                frames_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = cap.get(cv2.CAP_PROP_FPS)
                total_time = frames_cnt / fps
                if total_time < self.sequence_time:
                    self.skip_id.append(id_)
                    warn_message = 'Video {} has time {} less than sequence_time {} and will be skipped'.format(
                        id_,
                        total_time,
                        self.sequence_time
                    )
                    warnings.warn(ShortVideoWarning(warn_message))
    
    def __next_frame_step(self, fps):
        if self.fps is None:
            next_frame_step = 1
        else:
            next_frame_step = int(np.ceil(fps / self.fps))
        return next_frame_step
    
    def __make_start_positions(self):
        for id_ in self.ids:
            if id_ not in self.skip_id:
                video_path = os.path.join(self.videos, id_)
                with VideoStream(video_path) as cap:
                    frames_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    next_frame_step = self.__next_frame_step(fps)
                    start_positions = range(
                        0,
                        frames_cnt - self.timesteps * next_frame_step,
                        self.shift_frames * next_frame_step
                    )
                    for start_position in start_positions:
                        self.start_positions.append((id_, start_position))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.start_positions))
        if self.shuffle:
            if self.seed is not None:
                np.random.seed(self.seed)
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        length = int(np.ceil(len(len(self.start_positions)) / float(self.batch_size)))
        return length
    
    def __get_x(self, batch_start_positions):
        batch_x = []
        for id_, start_position in tqdm_notebook(batch_start_positions):
            video_path = os.path.join(self.videos, id_)
            with VideoStream(video_path) as cap:
                fps = cap.get(cv2.CAP_PROP_FPS)
                next_frame_step = self.__next_frame_step(fps)
                frame_sequence = []
                current_pos = start_position
                for i in range(self.timesteps):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, current_pos)
                    ret, frame = cap.read()
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame = cv2.resize(frame, self.target_size)
                    frame_sequence.append(frame)
                    current_pos += next_frame_step
                frame_sequence = np.array(frame_sequence)
            batch_x.append(frame_sequence)
        batch_x = np.array(batch_x)
        return batch_x
    
    def __get_y(self, batch_start_positions):
        batch_y = []
        for id_, start_position in batch_start_positions:
            index = self.ids.index(id_)
            batch_y.append(self.classes[index])
        batch_y = np.array(batch_y)
        batch_y = to_categorical(batch_y, num_classes=len(self.class_names))
        return batch_y
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_start_positions = [self.start_positions[i] for i in indexes]
        batch_x = self.__get_x(batch_start_positions)
        if self.fit_eval:
            batch_y = self.__get_y(batch_start_positions)
            return batch_x, batch_y
        return batch_x

In [238]:
tmp = ImageSequenceGenerator('../videos', train_ids, train_classes, ['0', '1'], seed=1992, fit_eval=False)